In [1]:
from collections import OrderedDict

import numpy as np
from qiskit import Aer
from qiskit.transpiler import PassManager

from qiskit_aqua import Operator, QuantumInstance
from qiskit_aqua.algorithms.adaptive import VQE
from qiskit_aqua.algorithms.classical import ExactEigensolver
from qiskit_aqua.algorithms.components.optimizers import L_BFGS_B
from qiskit_aqua.algorithms.components.variational_forms import RYRZ

from qiskit_aqua_chemistry import FermionicOperator
from qiskit_aqua_chemistry.drivers import ConfigurationManager

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'H .0 .0 .0; H .0 .0 0.735'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [3]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)
qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.to_matrix()
qubitOp.chop(10**-10)

In [4]:
# if you do not install any driver and want like to start with random Hamiltonian
# SIZE=4
# matrix = np.random.random((SIZE,SIZE))
# qubitOp = Operator(matrix=matrix)

In [5]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['eigvals'][0].real))

The exact ground state energy is: -1.857275030202381


In [6]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B()
lbfgs.set_options(maxfun=1000, factr=10, iprint=10)

# setup variation form generator (generate trial circuits for VQE)
var_form = RYRZ(qubitOp.num_qubits, 5, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = VQE(qubitOp, var_form, lbfgs, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -1.8558394075654778
Parameters: [-0.36421285 -3.14159265  0.40713821  0.628366    1.16855107  3.13654292
  3.04706761  1.01598668  3.14159265  0.01276409 -2.71605135 -0.55606055
  1.38614919  1.99367984  2.96826647  3.09819302  2.66399496  0.21704659
 -3.14159265  2.45685901  0.0471695   1.59525629 -0.0287444   0.24710631
 -3.14159265 -2.76470226 -0.5284518  -2.3109039   1.56821496  0.27591985
  2.92605829 -1.72430953  2.17302923 -0.59972485 -3.14159265  0.78339345
 -2.0963081  -1.43178347  1.61460406  3.11192051  2.9109565   1.92113649
 -2.71970626 -1.01656819 -3.13375028 -2.96831398 -1.59718039 -1.13423582]
